**What we need for the implementation**
- Relational Database Schema
    - R set of relations
    - K function that maps each relation to its set of primary key attributes
    - F function that maps each relation to its set of attributes, 
        where each attribute is given by its column name together with its datatype
- Each relation has:
    - Atribute set (F)
    - pk set (K)
    - Foreign key set (subset of F)
- Functions:
    - docTable: returns the textual description of a relation
    - docAttr: returns the textual description of an attribute a in relation r
- Ontology Representation:
    - C set of class identifiers (concepts)
    - P set of properties
        - Pobj subset of object properties
        - Pdata subset of data properties
    - A set of axioms 
        - class hierarchies, 
        - domain and range assertions, 
        - property characteristics
    - M set of annotation assertions
        - labels
        - comments
        - provenance info
- External Ontology Repo
    - DINGO
- Lexical view converter 
    - of any ontology
    - **CHECK**
- RAG pipeline 
    - Embed lexical views of 
        - external ontologies
        - Relational schema 
        - core ontology
    - Index in FAISS 
    - At each relation (...)